In [1]:
%load_ext autoreload
%autoreload 2    

# Setup

In [2]:
DB_NAME = "./memories.db"

In [3]:
from broai.prompt_management.core import PromptGenerator
from broai.prompt_management.interface import Persona, Instructions, Examples, Example
from pydantic import BaseModel, Field
from typing import List
from broai.experiments.bro_agent import BroAgent
import json
from broai.interface import Context, Contexts
from broai.experiments.vector_store import DuckVectorStore

/home/ec2-user/SageMaker/broai-researcher-assistant/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from package.jargon_store import JargonStore, JargonRecord

In [5]:
from broai.experiments.cross_encoder import ReRanker
rr = ReRanker()

/tmp/ipykernel_22259/2186289470.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: ReRanker
  rr = ReRanker()


In [6]:
from broai.experiments.huggingface_embedding import BAAIEmbedding, EmbeddingDimension
baai_em = BAAIEmbedding()

/tmp/ipykernel_22259/2466456318.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 195690.70it/s]


In [7]:
raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)
jargon_memory = JargonStore(db_name=DB_NAME, table="jargon_memory")

/tmp/ipykernel_22259/1440006260.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_22259/1440006260.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
/tmp/ipykernel_22259/1440006260.py:3: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)


# Agent Flows
- ContextEnricher
- JargonExtractor

In [8]:
from agents.context_enricher import ContextEnricher, InputContext
from agents.jargon_extractor import JargonExtractor, InputContext

In [9]:
ContextEnricher.run(InputContext(context="This is a context for BroAI."))

Summary(summary='Information about BroAI')

In [10]:
JargonExtractor.run(InputContext(context="This is a context for BroAI."))

/home/ec2-user/SageMaker/broai-researcher-assistant/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/ec2-user/SageMaker/broai-researcher-assistant/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the evidence of the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


Jargons(jargons=[Jargon(jargon='BroAI', evidence='This is a context for BroAI', explanation='BroAI is likely an artificial intelligence system or model')])

In [11]:
# import os
# os.remove(DB_NAME)

# Parsing markdown

In [12]:
from broai.experiments.chunk import split_markdown, consolidate_markdown, get_markdown_sections, split_overlap
from broai.interface import Context, Contexts

In [13]:
with open("./docs/storm.md", "r") as f:
    markdown_text = f.read()

In [14]:
chunks = split_markdown(markdown_text)

Markdown headings: max(4)


/tmp/ipykernel_22259/3854812085.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  chunks = split_markdown(markdown_text)


In [15]:
consolidated_chunks = consolidate_markdown(chunks)

/tmp/ipykernel_22259/1728333434.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  consolidated_chunks = consolidate_markdown(chunks)


In [45]:
consolidated_chunks[3]

'2 FreshWiki\n\nWe study generating Wikipedia-like articles from scratch, placing emphasis on the *pre-writing* stage [\\(Rohman,](#page-11-2) [1965\\)](#page-11-2), which involves the demanding sub-tasks of gathering and curating relevant information ("research"). This models the human\n\n<span id="page-1-0"></span><sup>1</sup>Our resources and code are released at [https://github.](https://github.com/stanford-oval/storm) [com/stanford-oval/storm](https://github.com/stanford-oval/storm).\n\n<span id="page-2-1"></span>\n\n|                            | Domain | Scope        | Given<br>Outline? | Given<br>Refs? |\n|----------------------------|--------|--------------|-------------------|----------------|\n| Balepur et al. (2023)      | One    | One para.    | /                 | Yes            |\n| Qian et al. (2023)         | All    | One para.    | /                 | No             |\n| Fan and Gardent (2022)     | One    | Full article | Yes               | No             |\n| Liu e

In [16]:
sections = get_markdown_sections(consolidated_chunks)

/tmp/ipykernel_22259/4124938538.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  sections = get_markdown_sections(consolidated_chunks)


In [41]:
sections

['# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024',
 'Abstract',
 '1 Introduction',
 '2 FreshWiki',
 '2.1 The FreshWiki Dataset',
 '2.2 Outline Creation and Evaluation',
 '3 Method',
 '3.1 Perspective-Guided Question Asking',
 '3.2 Simulating Conversations',
 '3.3 Creating the Article Outline',
 '3.4 Writing the Full-Length Article',
 '4 Experiments',
 '4.2 Automatic Metrics',
 '4.3 Baselines',
 '4.4 STORM Implementation',
 '5 Results and Analysis',
 '5.2 Ablation Studies',
 '6 Human Evaluation',
 '7 Related Works',
 '8 Conclusion',
 'Limitations',
 'Acknowledgements',
 'Ethics Statement',
 'References',
 'A Dataset Details',
 'B Pseudo Code of STORM',
 'C Automatic Evaluation Details',
 'C.2 LLM Evaluator',
 'C.3 More Discussion of the Citation Quality',
 'Algorithm 1: STORM',
 'D Human Evaluation Details',
 'E Error Analysis',
 'Interest Level',
 'Coherence and Organization',
 'Relevance and Focus',
 'Broad Coverage',
 'Verifiability',
 '**Taylor Hawkins**',
 '**# Early Life and Background**

In [17]:
contexts = Contexts()
source = ".docs/test1/storm.md"
for section, chunk in zip(sections, consolidated_chunks):
    contexts.add_context(Context(context=chunk, metadata={"section": section, "source": source, "type": "document"}))
len(contexts.contexts)

50

In [18]:
max_tokens = 500

new_contexts = split_overlap(contexts.contexts, max_tokens=max_tokens, overlap=int(max_tokens*0.3))
new_contexts = Contexts(contexts=new_contexts)
len(new_contexts.contexts)

/tmp/ipykernel_22259/2897930821.py:3: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap(contexts.contexts, max_tokens=max_tokens, overlap=int(max_tokens*0.3))


85

# Create: raw_memory

In [19]:
raw_memory.add_contexts(new_contexts.contexts)

In [20]:
raw_memory.read_all().head(3)

,id,context,metadata,type,embedding,created_at,updated_at
0,c337373e-ec24-45db-a58a-2cf5ce9f5f1f,# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024\n\nAss...,"{""section"":""# arXiv:2402.14207v2 [cs.CL] 8 Apr...",document,"[-0.04901123, -0.010894775, -0.02658081, -0.02...",2025-04-28 19:00:57.476582,2025-04-28 19:00:57.476582
1,e2fff818-1711-4ec6-aae0-52264d732026,Abstract\n\nWe study how to apply large langua...,"{""section"":""Abstract"",""source"":"".docs/test1/st...",document,"[0.0079193115, -0.016906738, -0.025924683, 0.0...",2025-04-28 19:00:57.476650,2025-04-28 19:00:57.476650
2,2ddeb820-4636-4564-9edd-6e99caffd2cf,1 Introduction\n\nLarge language models (LLMs)...,"{""section"":""1 Introduction"",""source"":"".docs/te...",document,"[-0.055419922, 0.009933472, -0.023742676, -0.0...",2025-04-28 19:00:59.550320,2025-04-28 19:00:59.550320


# Create: enrich_memory

In [21]:
%%time
from tqdm import tqdm
summaries = []
errors = []
for enum, context in enumerate(tqdm(new_contexts.contexts)):
    try:
        response = ContextEnricher.run(InputContext(context=context.context))
        summaries.append(
            Context(id=context.id, context=response.summary, metadata=context.metadata.copy())
        )
    except Exception as e:
        errors.append((enum, e, context))
len(summaries), len(errors)

100%|██████████| 85/85 [01:23<00:00,  1.02it/s]

CPU times: user 3.23 s, sys: 71.4 ms, total: 3.31 s
Wall time: 1min 23s


(85, 0)

In [22]:
enrich_memory.add_contexts(contexts=summaries)

In [23]:
enrich_memory.read_all().head(3)

,id,context,metadata,type,embedding,created_at,updated_at
0,c337373e-ec24-45db-a58a-2cf5ce9f5f1f,Researchers from Stanford University propose u...,"{""section"":""# arXiv:2402.14207v2 [cs.CL] 8 Apr...",document,"[-0.022033691, 0.005504608, -0.036621094, -0.0...",2025-04-28 19:01:30.417917,2025-04-28 19:01:30.417917
1,e2fff818-1711-4ec6-aae0-52264d732026,"Researchers propose STORM, a writing system to...","{""section"":""Abstract"",""source"":"".docs/test1/st...",document,"[-0.012672424, 0.017944336, -0.00819397, -0.01...",2025-04-28 19:01:31.413041,2025-04-28 19:01:31.413041
2,2ddeb820-4636-4564-9edd-6e99caffd2cf,The paper explores the challenges of using lar...,"{""section"":""1 Introduction"",""source"":"".docs/te...",document,"[-0.021621704, 0.00053167343, -0.017715454, -0...",2025-04-28 19:01:32.609151,2025-04-28 19:01:32.609151


# Create: longterm_memory

In [24]:
combo_contexts = []
for n, s in zip(new_contexts.contexts, summaries):
    if n.id == s.id:
        combo_contexts.append(
            Context(id=n.id, context=f"{s.context}\n\n{n.context}", metadata=n.metadata.copy())
        )


In [25]:
len(new_contexts.contexts), len(summaries), len(combo_contexts)

(85, 85, 85)

In [26]:
longterm_memory.add_contexts(contexts=combo_contexts)

In [27]:
longterm_memory.read_all().head(3)

,id,context,metadata,type,embedding,created_at,updated_at
0,c337373e-ec24-45db-a58a-2cf5ce9f5f1f,Researchers from Stanford University propose u...,"{""section"":""# arXiv:2402.14207v2 [cs.CL] 8 Apr...",document,"[-0.04071045, 0.0022087097, -0.026824951, -0.0...",2025-04-28 19:02:53.996452,2025-04-28 19:02:53.996452
1,e2fff818-1711-4ec6-aae0-52264d732026,"Researchers propose STORM, a writing system to...","{""section"":""Abstract"",""source"":"".docs/test1/st...",document,"[-0.006881714, -0.012702942, -0.025222778, -0....",2025-04-28 19:02:53.996487,2025-04-28 19:02:53.996487
2,2ddeb820-4636-4564-9edd-6e99caffd2cf,The paper explores the challenges of using lar...,"{""section"":""1 Introduction"",""source"":"".docs/te...",document,"[-0.034332275, -0.0070762634, -0.017120361, -0...",2025-04-28 19:02:53.996498,2025-04-28 19:02:53.996498


# Create: jargon_memory

In [28]:
%%time
from tqdm import tqdm
jargons_contexts = []
errors = []
for enum, context in enumerate(tqdm(new_contexts.contexts[:])):
    try:
        response = JargonExtractor.run(InputContext(context=context.context))
        metadata = context.metadata.copy()
        for j in response.jargons:
            if j.evidence.lower() != "none":
                jr = JargonRecord(jargon=j.jargon, evidence=j.evidence, explanation=j.explanation, metadata=metadata.copy())
                jargons_contexts.append(jr)
    except Exception as e:
        print(enum, "="*20)
        errors.append(
            (enum, str(e), context)
        )

  0%|          | 0/85 [00:00<?, ?it/s]/home/ec2-user/SageMaker/broai-researcher-assistant/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/ec2-user/SageMaker/broai-researcher-assistant/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the evidence of the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
 27%|██▋       | 23/85 [00:43<02:18,  2.24s/it]/home/ec2-user/SageMaker/br

Both parse_structured_output and content_extractor failed:
1 validation error for Jargons
  Invalid JSON: invalid escape at line 11 column 194 [type=json_invalid, input_value='\n{\n    "jargons": [\n ...\n        }\n    ]\n}\n', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
BroAgent.parse_structured_output() got multiple values for argument 'text'


100%|██████████| 85/85 [02:21<00:00,  1.66s/it]

CPU times: user 3.44 s, sys: 52.3 ms, total: 3.49 s
Wall time: 2min 21s


In [29]:
len(jargons_contexts), len(errors)

(182, 0)

In [30]:
jargons_contexts[0]

JargonRecord(id='462ccc4f-89db-4d48-b63b-45ec2861a4e9', jargon='arXiv', evidence='# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024', explanation='arXiv is an online archive of electronic preprints', metadata={'section': '# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024', 'source': '.docs/test1/storm.md', 'type': 'document', 'sequence': 0}, created_at='2025-04-28 19:02:59.013587')

In [31]:
jargon_memory.add_jargons(jargons=jargons_contexts)

In [32]:
jargon_memory.filter_metadata(section="1 Introduction")

,id,jargon,evidence,explanation,metadata,created_at,updated_at
0,1c620624-ccaf-4bbc-b38c-a2088ca9e935,LLMs,Large language models (LLMs) have demonstrated...,LLMs are large language models,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 19:03:02.162009,2025-04-28 19:03:02.162009
1,a67094d0-baf2-44e1-bf90-dbe1b0b83ac6,RAG,retrieval-augmented generation (*RAG*),RAG is short for retrieval-augmented generation,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 19:03:02.162427,2025-04-28 19:03:02.162427
2,07824b51-5ae3-42e1-9c7c-356b6ab6fe6c,STORM,we propose the STORM paradigm for the Synthesi...,STORM is short for Synthesis of Topic Outlines...,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 19:03:02.162489,2025-04-28 19:03:02.162489
3,b6c618f3-b506-4723-b36a-0eba8fdad2cf,RAG,current strategies often involve retrieval-aug...,RAG is short for retrieval-augmented generation,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 19:03:04.081610,2025-04-28 19:03:04.081610
4,707c5462-1465-431c-8937-71537bddbdff,LLMs,To endow LLMs with the capacity to conduct bet...,LLMs is short for Large Language Models,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 19:03:04.082050,2025-04-28 19:03:04.082050
5,8a50dbfa-a635-42d8-82de-aae082439bc9,STORM,we propose the STORM paradigm for the Synthesi...,STORM is short for Synthesis of Topic Outlines...,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 19:03:04.082071,2025-04-28 19:03:04.082071


In [33]:
jargon_memory.read_all()['jargon'].value_counts().head(20)

jargon
STORM                  28
LLM                    12
RAG                     9
LLMs                    8
ACL                     8
oRAG                    6
Wikipedia               5
R                       4
API                     4
GPT-3.5                 4
Mistral 7B-Instruct     4
EMNLP                   4
NLP                     4
dspy                    3
AI                      3
arXiv                   2
WikiSum                 2
GPT-4                   2
Likert scale            2
PMLR                    2
Name: count, dtype: int64

In [34]:
jargon_memory.fulltext_search(search_query="What does FreshWiki stand for?", context=False).to_dict(orient="records")

[{'id': '300ee810-3a17-4434-850e-f5e0b7f9ff8b',
  'jargon': 'FreshWiki',
  'evidence': 'we randomly select 100 samples from the FreshWiki dataset',
  'explanation': 'FreshWiki is a dataset',
  'metadata': '{"section":"4 Experiments","source":".docs/test1/storm.md","type":"document","sequence":13}',
  'created_at': Timestamp('2025-04-28 19:03:21.277912'),
  'updated_at': Timestamp('2025-04-28 19:03:21.277912'),
  'score': 2.1826533613212757}]

In [35]:
jargon_memory.fulltext_search(search_query="What does FreshWiki stand for?", context=True)

[JargonRecord(id='300ee810-3a17-4434-850e-f5e0b7f9ff8b', jargon='FreshWiki', evidence='we randomly select 100 samples from the FreshWiki dataset', explanation='FreshWiki is a dataset', metadata={'section': '4 Experiments', 'source': '.docs/test1/storm.md', 'type': 'document', 'sequence': 13}, created_at=Timestamp('2025-04-28 19:03:21.277912'))]

In [36]:
jargon_memory.fulltext_search(search_query="oRAG", context=False)

,id,jargon,evidence,explanation,metadata,created_at,updated_at,score
0,ca8ff634-a699-4911-8e81-bdc5222a36ee,oRAG,RAG/oRAG,"oRAG is likely a variant of RAG, retrieval aug...","{""section"":""4.4 STORM Implementation"",""source""...",2025-04-28 19:03:31.585878,2025-04-28 19:03:31.585878,1.516439
1,faa52b9d-8c11-495b-9b80-2874f4f8907b,oRAG,oRAG,oRAG is a variant of RAG,"{""section"":""4.4 STORM Implementation"",""source""...",2025-04-28 19:03:29.424699,2025-04-28 19:03:29.424699,1.516439
2,294c853c-8f89-43a9-a409-c94edc4a1260,oRAG,We randomly sample 20 topics from our dataset ...,oRAG is a baseline method for article generation,"{""section"":""6 Human Evaluation"",""source"":"".doc...",2025-04-28 19:03:48.891536,2025-04-28 19:03:48.891536,1.516439
3,bb7d1160-dfe9-462a-b5a8-33c3dff506c0,oRAG,*oRAG* significantly outperforms *RAG*,oRAG is a variant of RAG,"{""section"":""5 Results and Analysis"",""source"":""...",2025-04-28 19:03:37.376164,2025-04-28 19:03:37.376164,1.516439
4,47f39dd4-b7f3-4c5b-b1c3-9e1a4e5fb89c,oRAG,"Outline-driven RAG (oRAG), which is identical ...",oRAG is an outline-driven retrieval-augmented ...,"{""section"":""4.3 Baselines"",""source"":"".docs/tes...",2025-04-28 19:03:25.230515,2025-04-28 19:03:25.230515,1.516439


In [37]:
jargon_memory.fulltext_search(search_query="STORM", context=False)["explanation"].tolist()

['STORM is short for Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking',
 'STORM is short for Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking',
 'STORM is a system or method for discovering perspectives and controlling question asking process',
 'STORM is a system that automates the pre-writing stage',
 'STORM is a writing system for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking']

In [38]:
jargon_memory.fulltext_search(search_query="oRAG", context=False)["explanation"].tolist()

['oRAG is likely a variant of RAG, retrieval augmented generation',
 'oRAG is a variant of RAG',
 'oRAG is a baseline method for article generation',
 'oRAG is a variant of RAG',
 'oRAG is an outline-driven retrieval-augmented generation']

In [39]:
jargon_memory.fulltext_search(search_query="LLM", context=False).to_dict(orient="records")

[{'id': 'de1bce2f-5765-4f12-a023-510e0c504a7d',
  'jargon': 'LLMs',
  'evidence': 'modern LLMs are generally trained on Wikipedia text',
  'explanation': 'LLMs are large language models',
  'metadata': '{"section":"2.1 The FreshWiki Dataset","source":".docs/test1/storm.md","type":"document","sequence":6}',
  'created_at': Timestamp('2025-04-28 19:03:08.684056'),
  'updated_at': Timestamp('2025-04-28 19:03:08.684056'),
  'score': 0.9945755623058142},
 {'id': '1c620624-ccaf-4bbc-b38c-a2088ca9e935',
  'jargon': 'LLMs',
  'evidence': 'Large language models (LLMs) have demonstrated impressive writing capabilities',
  'explanation': 'LLMs are large language models',
  'metadata': '{"section":"1 Introduction","source":".docs/test1/storm.md","type":"document","sequence":2}',
  'created_at': Timestamp('2025-04-28 19:03:02.162009'),
  'updated_at': Timestamp('2025-04-28 19:03:02.162009'),
  'score': 0.9945755623058142},
 {'id': '18a5e64b-588b-4747-99d6-56b783487196',
  'jargon': 'LLM',
  'eviden

In [40]:
jargon_memory.fulltext_search(search_query="STORM", context=False)

,id,jargon,evidence,explanation,metadata,created_at,updated_at,score
0,8a50dbfa-a635-42d8-82de-aae082439bc9,STORM,we propose the STORM paradigm for the Synthesi...,STORM is short for Synthesis of Topic Outlines...,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 19:03:04.082071,2025-04-28 19:03:04.082071,0.84488
1,07824b51-5ae3-42e1-9c7c-356b6ab6fe6c,STORM,we propose the STORM paradigm for the Synthesi...,STORM is short for Synthesis of Topic Outlines...,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 19:03:02.162489,2025-04-28 19:03:02.162489,0.84488
2,d6b4696b-6574-41b3-9839-f5d1bbd812f2,STORM,STORM discovers different perspectives by surv...,STORM is a system or method for discovering pe...,"{""section"":""3.1 Perspective-Guided Question As...",2025-04-28 19:03:13.876676,2025-04-28 19:03:13.876676,0.84488
3,898caf7b-7ef3-4155-a8ed-d0d45adbd526,STORM,We present STORM to automate the pre-writing s...,STORM is a system that automates the pre-writi...,"{""section"":""3 Method"",""source"":"".docs/test1/st...",2025-04-28 19:03:12.026500,2025-04-28 19:03:12.026500,0.84488
4,05acc5d0-2ed8-4326-a0a5-d02be5eed257,STORM,"We propose STORM, a writing system for the Syn...",STORM is a writing system for the Synthesis of...,"{""section"":""Abstract"",""source"":"".docs/test1/st...",2025-04-28 19:03:00.348732,2025-04-28 19:03:00.348732,0.84488
